In [38]:
import pandas as pd
import numpy as np
import time
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import datetime
import itertools
from time import sleep

In [39]:
chrome_options = Options()
chrome_options.headless = False
# chrome_options.add_argument("--headless")
#driver = webdriver.Chrome(options=chrome_options)

In [40]:
Airlines = []
Aeroplane = []
Dept_time = []
Arrival_time = []
Source = []
Destination = []
Duration = []
Stops = []
Date = []
Price = []
Free_Meal = []
Checkin_Baggage = []
Taxes = []
Day_Track = []
Emission = []


In [41]:
start_date = datetime.date(2024,1,10)
end_date = datetime.date(2024,1,11)
delta = datetime.timedelta(days=1)
dates = [start_date+delta*i for i in range((end_date-start_date).days+1)]
cities = ["Mumbai","New", "Benga", "Kolk", "Chennai"]
routes = list(itertools.permutations(cities,2))

for date in dates:
    for route in routes:
        driver = webdriver.Chrome(options=chrome_options)
        dte = date.strftime("%d/%m/%Y")
        departure_city, arrival_city = route  # Unpack the tuple

        url = "https://www.yatra.com/"
        driver.get(url)
        time.sleep(2)
        #driver.maximize_window()
        from selenium.webdriver.common.keys import Keys
        wait = WebDriverWait(driver, 8)
        time.sleep(5)

        # iframe = WebDriverWait(driver, 10).until(
        #     EC.presence_of_element_located((By.ID, "webklipper-publisher-widget-container-notification-frame"))
        # )
        # driver.switch_to.frame(iframe)

        # # Now that we are in the iframe, locate the close button and click it
        # close_button = WebDriverWait(driver, 10).until(
        #     EC.element_to_be_clickable((By.CLASS_NAME, "close"))
        # )
        # close_button.click()

        # # Switch back to the main window
        # driver.switch_to.default_content()


        # driver.execute_script("arguments[0].click();", ad_close_button)
        depart_from = driver.find_element(By.XPATH, "//input[@id='BE_flight_origin_city']")
        depart_from.click()
        time.sleep(2)
        depart_from.send_keys(departure_city)
        time.sleep(2)
        depart_from.send_keys(Keys.ENTER)
        time.sleep(4)
        going_to = driver.find_element(By.XPATH, "//input[@id='BE_flight_arrival_city']")
        going_to.send_keys(arrival_city)
        going_to.click()
        time.sleep(2)
        going_to.send_keys(Keys.ENTER)
        time.sleep(4)
        wait = WebDriverWait(driver, 5)
        try:
            origin = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='BE_flight_origin_date']")))
            origin.click()

            all_dates = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@id='monthWrapper']//tbody//td[not(contains(@class, 'inActiveTD'))]")))
            for ddate in all_dates:
                if ddate.get_attribute("data-date") == dte:  # Verify the correct attribute
                    driver.execute_script("arguments[0].scrollIntoView(true);", ddate)
                    ddate.click()
                    break
        except TimeoutException:
            print("Element not found or not clickable.")
            
        time.sleep(3)

        # Code to click on the button with the ID 'BE_flight_fsearch_btn'
        search_button = driver.find_element(By.ID, "BE_flight_flsearch_btn")
        search_button.click()
        time.sleep(8)
        from tqdm import tqdm
        for _ in tqdm (range(0,50)):
            time.sleep(0.1)
            driver.execute_script("window.scrollBy(0,2000)","")
            time.sleep(0.2)
            driver.execute_script("window.scrollBy(0,-500)","")
        flights=driver.find_elements(By.XPATH,'//div[@class="flight-det table full-width clearfix"]')
        len(flights)
        flight_blocks = driver.find_elements(By.XPATH, '//div[@class="flightItem border-shadow pr"]')
        for flight in flights:
            Date.append(date.strftime("%d/%m/%Y"))


        # Loop over flight blocks for Airlines
        for block in flight_blocks:
               airline_elem = block.find_element(By.XPATH, './/div[@class="fs-13 airline-name no-pad col-8"]/span')
               Airlines.append(airline_elem.text.replace("\n", " ")) if airline_elem else Airlines.append("N/A")

        # Loop over flight blocks for Aeroplane
        for block in flight_blocks:
               aeroplane_elem = block.find_element(By.XPATH, './/p[@class="normal fs-11 font-lightestgrey no-wrap fl-no"]/span')
               Aeroplane.append(aeroplane_elem.text) if aeroplane_elem else Aeroplane.append("N/A")

        # Loop over flight blocks for Departure Time
        for block in flight_blocks:
              dept_time_elem = block.find_element(By.XPATH, './/div[@class="i-b pr"]')
              Dept_time.append(dept_time_elem.text) if dept_time_elem else Dept_time.append("N/A")

        # Loop over flight blocks for Arrival Time
        for block in flight_blocks:
              arrival_time_elem = block.find_element(By.XPATH, './/p[@class="bold fs-15 mb-2 pr time"]')
              Arrival_time.append(arrival_time_elem.text) if arrival_time_elem else Arrival_time.append("N/A")

        # Loop over flight blocks for Source
        for block in flight_blocks:
             source_elem = block.find_element(By.XPATH, '//div[@class="i-b col-4 no-wrap text-right dtime col-3"]/p')
             Source.append(source_elem.text) if source_elem else Source.append("N/A")

        # Loop over flight blocks for Destination
        for block in flight_blocks:
            dest_elem = block.find_element(By.XPATH, '//div[@class="i-b pdd-0 text-left atime col-5"]/p[2]')
            Destination.append(dest_elem.text) if dest_elem else Destination.append("N/A")

        # Loop over flight blocks for Duration
        for block in flight_blocks:
             duration_elem = block.find_element(By.XPATH, '//div[@class="stop-cont pl-13"]/p')
             Duration.append(duration_elem.text) if duration_elem else Duration.append("N/A")

        # Loop over flight blocks for Stops
        for block in flight_blocks:
             stop_elem = block.find_element(By.XPATH, "//div[@class='stop-cont pl-13']/div/span")
             Stops.append(stop_elem.text) if stop_elem else Stops.append("N/A")

        #count = 0
        #price = driver.find_elements(By.XPATH,'//div[@class="i-b tipsy fare-summary-tooltip fs-18"]')
        #for i in tqdm(price):
        #    Price.append(i.text.replace(',',''))
        #    count = count +1
        #else:
        #    count = count +1
        #    pass
        # Loop over flight blocks for Price
        for block in flight_blocks:
             price_elem = block.find_element(By.XPATH, '//div[@class="i-b tipsy fare-summary-tooltip fs-18"]')
             Price.append(price_elem.text.replace(',', '')) if price_elem else Price.append("0")

        for block in flight_blocks:
            emiss = block.find_element(By.XPATH, '//div[@class="text font-lightgrey ml-5 mt-2"]/span')
            if emiss:
                Emission_text = emiss.text
                Emission.append(Emission_text)
            else:
                Emission.append("0")   
        
        wait.until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class,"link-color fs-12 cursor-pointer") and contains(text(),"Flight Details")]')))

        view_details_buttons = driver.find_elements(By.XPATH, '//div[contains(@class,"link-color fs-12 cursor-pointer") and contains(text(),"Flight Details")]')

        for button in view_details_buttons:
            driver.execute_script("arguments[0].click();", button)
            wait.until(EC.visibility_of_element_located((By.XPATH, '//div[contains(@class, "flight-details")]')))


        for block in flight_blocks:
            meal_element = block.find_elements(By.XPATH, './/span[@class="meal"]')
            if meal_element:
                meal_text = meal_element[0].text 
                Free_Meal.append(meal_text)
            else:
                Free_Meal.append("Paid Meal")


        for block in flight_blocks:
            checkin_baggage = block.find_elements(By.XPATH, './/span[@class="font-lightgrey"]')
            if checkin_baggage:
                checkin_baggage_text = checkin_baggage[0].text  
                Checkin_Baggage.append(checkin_baggage_text) 
            else:
                Checkin_Baggage.append("0 kgs")


        ext_blocks = driver.find_elements(By.XPATH, '//div[@class="flight-details pr slide-down ovf-hidden full-width table"]')
        for block in ext_blocks:
            tax_el = block.find_elements(By.XPATH, './/td[@class="text-right"]')
            if tax_el:
                tax_el_text = tax_el[1].text 
                Taxes.append(tax_el_text) 
            else:
                
                Taxes.append("0")


        for block in flight_blocks:
            day = block.find_elements(By.XPATH, './/div[@class="bold fs-12 mb-5"]')
            if day:
                day_text = day[0].text
                Day_Track.append(day_text)
            else:
                Day_Track.append("0")

        driver.quit()


100%|██████████| 50/50 [00:17<00:00,  2.91it/s]


In [42]:
print(len(Airlines))
print(len(Aeroplane))
print(len(Dept_time))
print(len(Arrival_time))
print(len(Source))
print(len(Destination))
print(len(Duration))
print(len(Stops))
print(len(Date))
print(len(Price))
print(len(Free_Meal))
print(len(Checkin_Baggage))
print(len(Taxes))
print(len(Day_Track))
print(len(Emission))

340
340
340
340
340
340
340
340
340
340
340
340
340
340
340


In [43]:
Emission.append(0)

In [44]:
Flight = pd.DataFrame({
    "Airline": Airlines,
    'Aeroplane': Aeroplane,
    "Date": Date,
    "Departure_Time": Dept_time,
    "Arrival_Time": Arrival_time,
    "Source": Source,
    "Destination": Destination,
    "Stops": Stops,
    "Duration": Duration,
    "Price": Price,
    "Meal Option": Free_Meal,
    "Checkin Baggage Allowed": Checkin_Baggage,
    "Taxes": Taxes,
    "Day": Day_Track,
    "Emissions": Emission
})

Flight.head()

ValueError: All arrays must be of the same length

In [ ]:
Flight.to_csv('example.csv')

In [ ]:
Flight.shape

(8340, 14)